### Shared Word

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
lang_folder = "Italian"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language
lang_pair = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language
word_start = 0  # 0
word_end = 28  # 28 

In [4]:
Path(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/4-Shared Word/{lang_folder.capitalize()} {lang_pair.capitalize()}").mkdir(parents=True, exist_ok=True)

In [5]:
def remove_repetition(word_group):
    words = word_tokenize(word_group)
    word_unique = set(words)
    if len(word_unique) == 1:
        return "repetitive_word_group"
    else:
        return word_group

In [6]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,e,17674355
1,non,14565060
2,che,14355207
3,di,13837236
4,un,9088369
...,...,...
290541,disonorevolmente,5
290542,disonoreresti,5
290543,saffici,5
290544,benenoi,5


In [7]:
df_word = df_word_all.iloc[word_start:word_end,]
df_word

,word,frequency
0,e,17674355
1,non,14565060
2,che,14355207
3,di,13837236
4,un,9088369
5,la,9013615
6,il,8605133
7,a,8169261
8,per,6758245
9,è,5629444


In [8]:
lang_pair_list = glob.glob(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()}_And_{lang_pair.lower().capitalize()}*_All.xlsx")
lang_pair_list

['/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/Italian/Italian_And_Turkish (tur)_Etymologeek_Result_All.xlsx']

In [9]:
df_pair = pd.read_excel(f"{lang_pair_list[0]}")
df_pair

,dict_entry_main,turkish_word
0,zero,sıfır
1,cifrare,şifre
2,biglietto,bilet
3,ticket,bilet
4,champagne,şampanya
...,...,...
316,tiorba,torba
317,turbante,tülbent
318,vaso,vazo
319,vite,vida


In [10]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]

In [11]:
word_select = df_word["word"].values.tolist()

In [12]:
words = df_pair["dict_entry_main"].values.tolist()

In [13]:
word_select_set = set(word_select)
disable_word_set = set(disable_list)
words_set = set(words)

In [14]:
word_list = list(word_select_set.union(words_set.difference(disable_word_set)))

In [15]:
# 2 way  Not: 1 way is long and in Shared Word ReDe part 
data_kind_list = ["Two", "Three", "Four", "Five", "Sentence"]
for i in data_kind_list:
    sent_ngram_file = f"{i}" # Two, Three, Four, Five, Sentence
 
    if sent_ngram_file.lower() != "sentence":
        data_kind = f"{sent_ngram_file}gram"  # Twogram, Threegram, Fourgram, Fivegram
        folder_path = "N Gram"
    else:
        data_kind = "sentence"
        folder_path = "Sentence"

    df = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/{folder_path.capitalize()}/Merge/{data_kind.capitalize()}_Merge.csv")
    d_list  = df.iloc[:,0].values.tolist()

    resultlist = []
    manager = multiprocessing.Manager()
    resultlist = manager.list()
    
    def word_in_wordgroup(d_list):
        mergelist = []
        try:
            word = d_list.split()
        except:
            word = []
            #pass  disabled for non split value
        var1 = range(len(word))
        for j in var1:
            if word[j] in word_list:
                mergelist.append(word[j])
                if len(mergelist) == len(word):
                        resultlist.append(d_list)
                            
    if __name__ == '__main__':
        # with Pool(16) as p:
        with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
            p.map(word_in_wordgroup, d_list) # string_word liste

    result_list = list(resultlist)
    df_result = pd.DataFrame(result_list, columns=[0])  # add columns parameter for empty result
    df_result = df_result.rename(columns = {0: f"{data_kind.lower()}"})
    df_merge = pd.merge(df_result, df, how="left", on=f"{data_kind.lower()}")
    df_merge_result = df_merge.sort_values(by="frequency", ascending=False)
    df_merge_result.drop_duplicates(inplace=True)
    df_merge_result[f"{data_kind.lower()}"] = df_merge_result[f"{data_kind.lower()}"].apply(lambda x: remove_repetition(x))  # added remove_repetition 
    df_merge_result = df_merge_result[df_merge_result[f"{data_kind.lower()}"] != "repetitive_word_group"]  # remove_repetition part
    df_merge_result.reset_index(drop=True, inplace=True)

    df_merge_result.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_{data_kind.capitalize()}.xlsx", index=False)
    

#### Concat Result

In [16]:
df_word = pd.DataFrame(word_list, columns=["word"])
df_word

,word
0,galion
1,orchestrale
2,sbandato
3,tipicamente
4,gazzetta
...,...
338,lanterna
339,mandarino
340,caramella
341,sistemistica


In [17]:
df_shared_twogram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Twogram.xlsx")
df_shared_twogram = df_shared_twogram.loc[:,["twogram","frequency"]]
df_shared_twogram.rename(columns={"frequency":"freq_twogram"}, inplace=True)
df_shared_twogram

,twogram,freq_twogram
0,non lo,427268
1,non e,366791
2,e un,326508
3,non è,279045
4,l ho,274284
...,...,...
5217,tavola si,3
5218,che vaso,3
5219,cancro sono,3
5220,cancro ti,3


In [18]:
df_shared_threegram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Threegram.xlsx")
df_shared_threegram = df_shared_threegram.loc[:,["threegram","frequency"]]
df_shared_threegram.rename(columns={"frequency":"freq_threegram"}, inplace=True)
df_shared_threegram

,threegram,freq_threegram
0,non l ho,44759
1,e una cosa,27545
2,non e un,26866
3,no non lo,22404
4,no non e,20218
...,...,...
14179,e controllo l,3
14180,e controllo il,3
14181,sono video in,3
14182,sono è una,3


In [19]:
df_shared_fourgram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Fourgram.xlsx")
df_shared_fourgram = df_shared_fourgram.loc[:,["fourgram","frequency"]]
df_shared_fourgram.rename(columns={"frequency":"freq_fourgram"}, inplace=True)
df_shared_fourgram

,fourgram,freq_fourgram
0,e una cosa che,5469
1,no non l ho,3504
2,no non lo e,3254
3,è una cosa che,2973
4,no no no non,2941
...,...,...
10778,una cosa di baseball,3
10779,una cosa e in,3
10780,una cosa e ho,3
10781,una cosa non ho,3


In [20]:
df_shared_fivegram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Fivegram.xlsx")
df_shared_fivegram = df_shared_fivegram.loc[:,["fivegram","frequency"]]
df_shared_fivegram.rename(columns={"frequency":"freq_fivegram"}, inplace=True)
df_shared_fivegram

,fivegram,freq_fivegram
0,e una cosa che non,542
1,no no no no non,499
2,è una cosa che non,414
3,non e una cosa che,412
4,no che non lo e,279
...,...,...
3107,e a cosa mi e,3
3108,e un elicottero che la,3
3109,e un cosa ma una,3
3110,e un biglietto per una,3


In [21]:
df_shared_sentence = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Sentence.xlsx")
df_shared_sentence = df_shared_sentence.loc[:,["sentence","frequency"]]
df_shared_sentence.rename(columns={"frequency":"freq_sentence"}, inplace=True)
df_shared_sentence

,sentence,freq_sentence
0,che cosa,37271
1,per cosa,17225
2,ma che,7802
3,di cosa,7553
4,lo sono,5145
...,...,...
2974,si e mi,3
2975,la scuola di etichetta,3
2976,è il giapponese,3
2977,si di astronomia,3


In [22]:
df_shared_all = pd.concat([df_word, df_shared_twogram, df_shared_threegram, df_shared_fourgram, df_shared_fivegram, df_shared_sentence], axis=1)
df_shared_all

,word,twogram,freq_twogram,threegram,freq_threegram,fourgram,freq_fourgram,fivegram,freq_fivegram,sentence,freq_sentence
0,galion,non lo,427268.0,non l ho,44759,e una cosa che,5469.0,e una cosa che non,542.0,che cosa,37271.0
1,orchestrale,non e,366791.0,e una cosa,27545,no non l ho,3504.0,no no no no non,499.0,per cosa,17225.0
2,sbandato,e un,326508.0,non e un,26866,no non lo e,3254.0,è una cosa che non,414.0,ma che,7802.0
3,tipicamente,non è,279045.0,no non lo,22404,è una cosa che,2973.0,non e una cosa che,412.0,di cosa,7553.0
4,gazzetta,l ho,274284.0,no non e,20218,no no no non,2941.0,no che non lo e,279.0,lo sono,5145.0
...,...,...,...,...,...,...,...,...,...,...,...
14179,NaN,NaN,NaN,e controllo l,3,NaN,NaN,NaN,NaN,NaN,NaN
14180,NaN,NaN,NaN,e controllo il,3,NaN,NaN,NaN,NaN,NaN,NaN
14181,NaN,NaN,NaN,sono video in,3,NaN,NaN,NaN,NaN,NaN,NaN
14182,NaN,NaN,NaN,sono è una,3,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
df_shared_all.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Result_With_Frequency.xlsx", index=False)

In [24]:
def word_in_twogram(df, list_column, target_column):

    '''word_in_twogram(df, "word", "twogram"):
       df is dataframe, list_column and target_column are 
       dataframe column string name. list_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_word_result = pd.DataFrame()
    for i in df[f"{list_column}"].dropna():
        try:
            word_in_twogram = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)] 
        except:
            pass        
        word_in_twogram.insert(0,"word",i)
        df_word_result = pd.concat([df_word_result,word_in_twogram], axis=0)
    df_word_result.reset_index(drop=True, inplace=True)

    return df_word_result

In [25]:
df_word_order_twogram = word_in_twogram(df_shared_all, "word", "twogram")
df_word_order_threegram = word_in_twogram(df_shared_all, "word", "threegram") 
df_word_order_fourgram = word_in_twogram(df_shared_all, "word", "fourgram") 
df_word_order_fivegram = word_in_twogram(df_shared_all, "word", "fivegram")
df_word_order_sentence = word_in_twogram(df_shared_all, "word", "sentence") 

In [26]:
df_word_order_join_twogram = df_word_order_twogram.groupby(["word"])["twogram"].apply(", ".join).reset_index()   # df_word_order_11.groupby(["word"])[f"twogram_pair_{lang_pair.lower()}"].transform(lambda x: ','.join(x))
df_word_order_join_threegram = df_word_order_threegram.groupby(["word"])["threegram"].apply(", ".join).reset_index()
df_word_order_join_fourgram = df_word_order_fourgram.groupby(["word"])["fourgram"].apply(", ".join).reset_index()
df_word_order_join_fivegram = df_word_order_fivegram.groupby(["word"])["fivegram"].apply(", ".join).reset_index()
df_word_order_join_sentence = df_word_order_sentence.groupby(["word"])["sentence"].apply(", ".join).reset_index()

In [27]:
dfs = [df_word_order_join_twogram,df_word_order_join_threegram,df_word_order_join_fourgram,df_word_order_join_fivegram,df_word_order_join_sentence]

In [28]:
df_word_order_join_all = reduce(lambda  left,right: pd.merge(left,right, on=['word'], how='outer'), dfs)  # left,right make left to right merge
#df_word_order_join_all = reduce(lambda  right,left: pd.merge(left,right, on=['word'], how='outer'), dfs)  # right,left make right to left merge
df_word_order_join_all  

,word,twogram,threegram,fourgram,fivegram,sentence
0,a,"a che, e a, a cosa, a un, sono a, a scuola, a ...","no sono a, ha a che, a cosa ti, una cosa a, a ...","cosa ha a che, no no sono a, cosa ho a che, co...","con cosa ho a che, e cosa ha a che, che cosa h...","a cosa, a scuola, a ha, a tavola, a che cosa, ..."
1,abbozza,abbozza l,NaN,NaN,NaN,NaN
2,ace,"di ace, e ace, ace e, sono ace, ace che, ace è...","con un ace, ace per il, ace e il, ace è una, a...","no no no ace, e con ace che, con un ace a",NaN,"e ace, l ace, la ace, è ace, no no no ace"
3,agente,"l agente, un agente, agente di, agente a, agen...","sono l agente, sono un agente, e l agente, un ...","sono un agente di, e un agente di, non sono un...","l agente che l ha, non sono un agente di, l ag...","l agente, sono un agente, un agente, no agente..."
4,agosto,"di agosto, in agosto, agosto e, e agosto, da a...","agosto e l, di agosto e, agosto non sono, agos...","di agosto e l, di agosto non sono, è da agosto...",NaN,"in agosto, e agosto, da agosto"
...,...,...,...,...,...,...
264,zero,"da zero, a zero, di zero, zero per, e zero, ze...","a zero per, zero a zero, da zero a, zero per i...","a zero per i, le a sono zero, zero a zero da, ...","ha una politica di zero, lo zero che ti ho, pe...","da zero, zero a zero, a zero, no zero, e zero,..."
265,zinco,"di zinco, e zinco, zinco e, lo zinco, in zinco...","di zinco e, di zinco ma, di zinco per, e di zi...","e di zinco per, e lo zinco sono",NaN,e zinco
266,zingaro,"lo zingaro, zingaro di, da zingaro, zingaro e,...","e lo zingaro, che lo zingaro, lo zingaro di, z...",ma lo zingaro no,NaN,"lo zingaro, e lo zingaro, ma lo zingaro no"
267,zucchero,"di zucchero, lo zucchero, e zucchero, zucchero...","con lo zucchero, di zucchero e, e lo zucchero,...","di zucchero e la, non con lo zucchero, lo zucc...","non ho lo zucchero per, la rosa di zucchero è,...","lo zucchero, e zucchero, con lo zucchero, e lo..."


In [29]:
df_word_order_join_all.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Join_Result_Without_Frequency.xlsx", index=False)

#### Copy Move And Delete

In [30]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared*.xlsx")
output_file

['Italian_Turkish_Shared_Twogram.xlsx',
 'Italian_Turkish_Shared_Threegram.xlsx',
 'Italian_Turkish_Shared_Fourgram.xlsx',
 'Italian_Turkish_Shared_Fivegram.xlsx',
 'Italian_Turkish_Shared_Sentence.xlsx',
 'Italian_Turkish_Shared_Result_With_Frequency.xlsx',
 'Italian_Turkish_Shared_Join_Result_Without_Frequency.xlsx']

In [31]:
for k in output_file:
    source = k # source directory
    destination = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/4-Shared Word/{lang_folder.capitalize()} {lang_pair.capitalize()}"
    shutil.copy2(source, destination)

In [32]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass